In [5]:
!pip install pillow
!pip install google.cloud
!pip install google-cloud-vision


import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="ocean-engineering-96e0c62d5fe3.json"

from google.cloud import vision

import PIL 

import io
import os
import cv2
import glob

from PIL import Image, ImageDraw, Image, ImageFont
import pandas as pd
def createCroppedImage(pillow_image, bounding, image_size, i,  caption='', confidence_score=0):
    global countImages
    width, height = image_size

    print(width, height)
    print('x0',bounding.normalized_vertices[0].x * width )
    print('y0',bounding.normalized_vertices[0].y * height )

    print('x1',bounding.normalized_vertices[1].x * width )
    print('y1',bounding.normalized_vertices[1].y * height )

    print('x2',bounding.normalized_vertices[2].x * width )
    print('y2',bounding.normalized_vertices[2].y * height )

    print('x3',bounding.normalized_vertices[3].x * width )
    print('y3',bounding.normalized_vertices[3].y * height )

    area = (bounding.normalized_vertices[0].x * width, 
              bounding.normalized_vertices[0].y * height,
              bounding.normalized_vertices[2].x * width,
              bounding.normalized_vertices[2].y * height)
    cropped_img = pillow_image.crop(area)
    pathCaption = 'Labeled Cropped Images/'+caption+'/'


    if not os.path.isdir(pathCaption):
        os.mkdir(pathCaption)
        countImages[caption] = 0

    print(pathCaption+caption+'_'+str(countImages[caption])+'.png')

    cropped_img.save(pathCaption+caption+'_'+str(countImages[caption])+'.png')

    countImages[caption] += 1

def draw_borders(pillow_image, bounding, color, image_size, caption='', confidence_score=0):

    width, height = image_size
    draw = ImageDraw.Draw(pillow_image)
    draw.polygon([
        bounding.normalized_vertices[0].x *
        width, bounding.normalized_vertices[0].y * height,
        bounding.normalized_vertices[1].x *
        width, bounding.normalized_vertices[1].y * height,
        bounding.normalized_vertices[2].x *
        width, bounding.normalized_vertices[2].y * height,
        bounding.normalized_vertices[3].x * width, bounding.normalized_vertices[3].y * height], fill=None, outline=color)

    #TODO: Validation needed
    font_size = width * height // 22000 if width * height > 400000 else 12

    #font = ImageFont.truetype(r'C:/Users/jiejenn/AppData/Local/Microsoft/Windows/Fonts/opensans-regular.ttf', 22)

    draw.text((bounding.normalized_vertices[0].x * width,
               bounding.normalized_vertices[0].y * height), text=caption, fill=color)

    # insert confidence score
    draw.text((bounding.normalized_vertices[0].x * width, bounding.normalized_vertices[0].y *
               height + 20), text='Confidence Score: {0:.2f}%'.format(confidence_score), fill=color)

    return pillow_image


import random
def draw_boundaries(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    pillow_image = Image.open(path)
    df = pd.DataFrame(columns=['name', 'score'])

    print('Number of objects found: {}'.format(len(objects)))
    i = 0

    for obj in objects:
        createCroppedImage(pillow_image, obj.bounding_poly, pillow_image.size, i, obj.name, obj.score)
        i+= 1
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')

        df = df.append(dict(name=object_.name, score=object_.score),ignore_index=True)

        r, g, b = random.randint(150, 255), random.randint(150, 255), random.randint(150, 255)

        draw_borders(pillow_image, object_.bounding_poly, (r, g, b), pillow_image.size, object_.name, object_.score)
        
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))
    print(df)
    #pillow_image.save('/content/attempt4.png')
    #pillow_image.show()
    pillow_image



inputPath = 'output1'
countImages = {}
for f in os.listdir(inputPath):
    if (f.endswith('.png') or f.endswith('.jpeg') or f.endswith('.jpg')):
        draw_boundaries(os.path.join(inputPath, f))

print('Done')


Number of objects found: 2
3840 2160
x0 1339.3597412109375
y0 383.70028853416443
x1 2058.8582611083984
y1 383.70028853416443
x2 2058.8582611083984
y2 1348.3519220352173
x3 1339.3597412109375
y3 1348.3519220352173
Labeled Cropped Images/Dog/Dog_0.png
3840 2160
x0 952.2823905944824
y0 888.5041165351868
x1 1641.8548965454102
y1 888.5041165351868
x2 1641.8548965454102
y2 1672.75381565094
x3 952.2823905944824
y3 1672.75381565094
Labeled Cropped Images/Dog/Dog_1.png

Dog (confidence: 0.9695957899093628)
Normalized bounding polygon vertices: 
 - (0.34879159927368164, 0.17763902246952057)
 - (0.5361610054969788, 0.17763902246952057)
 - (0.5361610054969788, 0.6242370009422302)
 - (0.34879159927368164, 0.6242370009422302)

Dog (confidence: 0.9409712553024292)
Normalized bounding polygon vertices: 
 - (0.2479902058839798, 0.4113444983959198)
 - (0.42756637930870056, 0.4113444983959198)
 - (0.42756637930870056, 0.7744230628013611)
 - (0.2479902058839798, 0.7744230628013611)
  name     score
0  Dog